### Read .dat file generated from box and confirm impure probabilities

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from banditpy.core import Bandit2Arm

# fp = Path("D:/Data/mab/BGM1-2025-08-18-14-57-47.dat")

basedir = Path("D:/Data/mab/BGdataset")
animals = [basedir / "BGM0", basedir / "BGM1", basedir / "BGM2"]

tasks: list[Bandit2Arm] = []
for folder in animals:
    dat_files = sorted(folder.glob("*.dat"))

    data = []
    for fp in dat_files:
        data.append(pd.read_csv(fp, sep=",", header=None))
    data = pd.concat(data, ignore_index=True)
    # data_prob = data[data[0] == 83].reset_index(drop=True)[4].to_numpy()
    # data_prob = data_prob.reshape(-1, 2)
    # prob_sum = data_prob.sum(axis=1)
    port1_idx = data[(data[0] == 83) & (data[1] == 1)].index
    p1 = data.loc[port1_idx[0], 4]
    p2 = data.loc[port1_idx[0] + 1, 4]
    session_id = 1
    behav_data_df = []
    for idx in range(port1_idx[0] + 2, len(data)):

        if data.loc[idx, 0] == 83 and data.loc[idx, 1] == 1:
            p1 = data.loc[idx, 4]
            session_id += 1
        if data.loc[idx, 0] == 83 and data.loc[idx, 1] == 2:
            p2 = data.loc[idx, 4]
        if data.loc[idx, 0] == 81 and data.loc[idx, 1] == 1:
            port = 1
        if data.loc[idx, 0] == 81 and data.loc[idx, 1] == 2:
            port = 2
        if data.loc[idx, 0] == 52:
            if port > 0:
                behav_data_df.append([port, 1, p1, p2, session_id])
                port = 0
        if data.loc[idx, 0] == -51:
            if port > 0:
                behav_data_df.append([port, 0, p1, p2, session_id])
                port = 0
        if data.loc[idx, 0] == -52:
            if port > 0:
                behav_data_df.append([port, -1, p1, p2, session_id])
                port = 0

    behav_data_df = np.array(behav_data_df)
    behav_data_df = pd.DataFrame(
        behav_data_df, columns=["port", "reward", "p1", "p2", "session_id"]
    )

    tasks.append(
        Bandit2Arm(
            probs=behav_data_df[["p1", "p2"]].to_numpy(),
            choices=behav_data_df["port"].to_numpy(),
            rewards=behav_data_df["reward"].to_numpy(),
            session_ids=behav_data_df["session_id"].to_numpy(),
        )
    )

In [2]:
import matplotlib.pyplot as plt
from banditpy.plots import plot_trial_by_trial_2Arm
from neuropy import plotting
from scipy import stats

fig = plotting.Fig(5, 4)
animal_names = ["BGM0", "BGM1", "BGM2"]

for i, task in enumerate(tasks):

    probs = task.probs[task.is_session_start, :]
    probs_corr = stats.pearsonr(probs[:, 0], probs[:, 1])
    print(probs_corr[0])

    p1_bins = np.linspace(0, 0.9, 10) + 0.05
    p2_bins = np.linspace(0, 0.9, 10) + 0.05
    H, xedges, yedges, _ = stats.binned_statistic_2d(
        probs[:, 0],
        probs[:, 1],
        values=probs[:, 0],
        statistic="count",
        bins=[p1_bins, p2_bins],
    )

    perf = task.filter_by_trials(
        min_trials=30, clip_max=100
    ).get_optimal_choice_probability()

    ax = fig.subplot(fig.gs[i, 0])
    ax.plot(perf, color="k")
    # task.plot_trial_by_trial(ax=ax)
    ax.set_ylim(0.1, 0.7)
    ax.set_xlabel("Trial_id")
    ax.set_ylabel("Choice (High)")
    ax.set_title(f"{animal_names[i]}'s performance")

    ax2 = fig.subplot(fig.gs[i, 1])
    cplot = ax2.pcolormesh(
        xedges,
        yedges,
        H.T,
        cmap="hot",
        vmin=0,
        vmax=14,
        shading="auto",
    )
    cb = plt.colorbar(cplot, ax=ax2, shrink=0.5)
    # ax2.imshow(prob_mat)
    ax2.set_xlabel("Prob1")
    ax2.set_ylabel("Prob2")
    ax2.set_title(f"{animal_names[i]}'s probability combinations")
    ax2.set_xticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    ax2.set_yticks([0.2, 0.3, 0.4, 0.6, 0.7, 0.8])
    cb.set_label("Counts")

# plt.plot(perf)
# plot_trial_by_trial_2Arm(task)

-0.16784866844083918
-0.7806018060170636
-0.24062800961813463
